In [ ]:
# Parse bed file and extract chrom, start, end positions of peaks
def parse_bed_file(peak_file):
    peaks = []
    with open(peak_file, 'r') as f:
        for line in f:
            if "#" in line:
                continue
            fields = line.strip().split('\t')
             # Change fields to 0, 1, 2 for BED file; 1, 2, 3 is for HOMER files (since I tested the functions using Lab 5 HOMER files)
            chrom = fields[1]
            start = int(fields[2])
            end = int(fields[3])
            peaks.append((chrom, start, end))
    return peaks

In [ ]:
parse_bed_file('peaks.txt')

In [ ]:
# Self implementation of extracting sequence from genome fasta file based on peak coordinates; VERY SLOW
def extract_sequence(chrom, start, end, genome_fasta):
    sequence = ""
    target_chrom = False
    with open(genome_fasta, 'r') as f:
        for line in f:
            if len(sequence) >= end:
                break
            if ">" + chrom in line and not target_chrom:
                target_chrom = True
                continue
            if target_chrom:
                sequence += line.strip()
    return sequence[start:end]

In [ ]:
foo = extract_sequence('Y', 100, 200, 'GRCm38.fa')    
foo

In [53]:
# Organizes the genome fasta file by chromosome so that we can later find sequences more quickly; MUCH FASTER
from Bio import SeqIO

def get_genome_info(fasta_file):
    genome_info = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        genome_info[record.id] = record.seq
    return genome_info

In [54]:
get_genome_info('GRCm38.fa')

{'1': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '10': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '11': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '12': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '13': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '14': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '15': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '16': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '17': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '18': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '19': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '2': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '3': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 '4': Seq('NNNN

In [55]:
# Extract sequence from organized genome fasta file based on peak coordinates; FAST
def extract_sequence(chrom, start, end, genome_info):
    return genome_info[chrom][start:end]

In [56]:
# Combining all functions to get peak sequences
def get_peak_sequences(bed_file, genome_fasta_file):
    peaks = parse_bed_file(bed_file) # Get peak coordinates
    genome_info = get_genome_info(genome_fasta_file) # Organize genome fasta file for fast sequence extraction
    sequences = [] # Store peak sequences
    for peak in peaks: # Extract sequences for each peak
        chrom, start, end = peak # Unpack peak coordinates
        sequence = extract_sequence(chrom, start, end, genome_info) # Extract sequence
        sequences.append(sequence) # Store sequence
    return sequences # Return all sequences

In [57]:
get_peak_sequences('peaks.txt', 'GRCm38.fa')

[Seq('TAGGGTGCGTTGGGCGGTGGCCGGGTATAAAAGACTCCGCCCGAGCGGGCGGCC...TTA'),
 Seq('GTTGGTTCCTGACTGTGCTCGGAGGAATGTTAATTAGCCTCTCCCTTCCGGAGG...CTT'),
 Seq('TCTCACCGTCAGGGGGAAGGGCGTCCGTCTGACCGCGAATAATAAATAAGAGCT...GCA'),
 Seq('TTGAGAGTTCTGGGCAGACGGCAGATGCATAACAAAGGTGCATGATAGCTCTGC...TGG'),
 Seq('TCTGATGGGAAGCCCGGGGGCTTGGGGTGTGTTCTGATGAGAGGCCTGGGGGCT...GGA'),
 Seq('GACTGGCATAGGGCAGAGCCTAGGTGGAGCAAGGGCGTGGTCAAAGACTTGCGG...GCG'),
 Seq('ACAGGGTGTGGCTGACTGGAACAGAAGGATACTTGTAGCTCTCTAGCGCCCCCT...CCG'),
 Seq('CCTGCTTCTGCCTGGAACAATGGGGTTAAAGGCACACACCTTCGCGCTCAGCTC...ATA'),
 Seq('CCTTGAGAGAGGAATGTGCGCATTCAGTGCTGTGCCTGTCCCCACCCAGGTTAC...TCC'),
 Seq('CTTCAGCCCCATATTTAGCTGCCTTCTCACCTCCCACGCCTGCTCGCCACCTTC...TCG'),
 Seq('CATCATCTGGTTGTCCCCTGTGGAGCAATACTGCCGCTCAGTGGATGAATGTGG...TGT'),
 Seq('CAGCCTAGAATAGAGCAAGGTTAGGCCCAGGTGTAATACAAATGGTAATTTCAG...TGG'),
 Seq('GGCAATGTGGGTGTGTCCCAGTATTTTTAATTGCTAATGGCTGTGCTGGCAGAG...CTC'),
 Seq('ATTATCAAACAGGCTTAATTCTCAGACTGAATGAAAATGGAAACAGGAGAAAGC...TCC'),
 Seq('GTCAGCCTCTATAA